<a href="https://colab.research.google.com/github/satyambhatt5/Deep_Learning-/blob/main/Waste_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Neural Network with TensorFlow and KERAS
  
import tensorflow as tf
import os
import cv2
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras import backend as K
import tensorflow as tf
 
from tensorflow.keras import layers

color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Practice Data Set/recycling waste"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
idg= ImageDataGenerator(rotation_range=30,rescale=1/255,brightness_range=(0.2,0.5),zoom_range=0.2,shear_range=0.2,horizontal_flip=True)



In [ ]:
train_gen= idg.flow_from_directory(directory="/content/drive/MyDrive/Colab Notebooks/Practice Data Set/recycling waste",target_size=(224,224),batch_size=128,class_mode="categorical")

In [ ]:
#let create the model here i am going use the vgg16 model 

from tensorflow.keras.applications.vgg16 import VGG16






In [ ]:
#download the model 

Vgg16=VGG16(include_top=False,input_shape=(224,224,3),weights="imagenet")



In [ ]:
# i dont want to make the train the model so the we will remove the model

for layer in Vgg16.layers:
  layer.trainable = False

In [ ]:
for layer in Vgg16.layers:
  print(layer.name,layer.trainable)

In [ ]:
#functional way two build the the model API

vgg_output=Vgg16.output

flatten_output=tf.keras.layers.Flatten()(vgg_output)

#first dense layer 
dense1=tf.keras.layers.Dense(256,activation='relu')(flatten_output)
bn1=tf.keras.layers.BatchNormalization()(dense1)
dp1=tf.keras.layers.Dropout(0.3)(bn1)

#second dense layer 
dense2=tf.keras.layers.Dense(128,activation='relu')(dp1)
dp2=tf.keras.layers.Dropout(0.2)(dense2)
dense3=tf.keras.layers.Dense(units=9,activation='softmax')(dp2)


In [ ]:
#del finalModel

In [ ]:
finalModel=tf.keras.models.Model(inputs=[Vgg16.input],outputs=[dense3])
finalModel.summary()

In [ ]:
finalModel.compile(optimizer=SGD(learning_rate=.01),loss=tf.keras.losses.categorical_crossentropy,metrics=["acc"])


In [ ]:
result=finalModel.fit_generator(train_gen,epochs=5,steps_per_epoch=len(train_gen))

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ["batteries", "clothes", "e-waste", "glass", "light blubs", "metal", "organic", "paper", "plastic"]
def waste_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (224,224))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value, " with ", predicted_accuracy, " % accuracy")